In [ ]:

import numpy as np

# load text data

#INPUT DATA =======================================

txt_data = "abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz " 
#txt_data = "hellohello hellohello hellohello hellohello hellohello " 
#txt_data = "hello hello hello hello hello " 
#txt_data = open('input.txt', 'r').read() # test external files

chars = list(set(txt_data)) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(txt_data)

# You can see the number of unique characters in your input data.
print("unique characters : ", num_chars)
print("txt_data_size : ", txt_data_size)

# ONE HOT ENCODE ==================================

# "enumerate" retruns index and value. Convert it to dictionary
char_to_int = dict((c, i) for i, c in enumerate(chars)) 
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
# "integer_encoded" is a list which has a sequence 
# converted from an original data to integers.
integer_encoded = [char_to_int[i] for i in txt_data] 
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

# Not actually used. ========================

onehot_encoded = []

for ix in integer_encoded: # ix is an index mapped to a unique character.
# A list len is equal to the number of unique characters and whose elements are all zero.
    letter = [0 for _ in range(len(chars))] 
    letter[ix] = 1 # 'letter' is a one-hot vector.
    onehot_encoded.append(letter) # Add a 1d list(a vector for one character).
onehot_encoded = np.array(onehot_encoded) # list to np-array   

print(onehot_encoded.shape)     #  = (len(data),len(chars))
print(onehot_encoded)

# invert encoding
# "argmax" returns the index of the largest value.
inverted = int_to_char[np.argmax(onehot_encoded[0])] 
print(inverted)

# INITIALIZATION =======================================

# hyperparameters

iteration = 5000
sequence_length = 15
#sequence_length =5 
batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
#hidden_size = 100  # size of hidden layer of neurons.  
hidden_size = 10  # size of hidden layer of neurons.  
learning_rate = 0.5e-1


# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1)

#FORWARD PROPAGATION =====================================

def forwardprop(inputs, targets, h_prev):
        
    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, 
        # but all of the elements are positive through the exp() function.
 
        # softmax (cross-entropy loss). Efficient and simple code
        loss += -np.log(ps[t][targets[t],0]) 

#         y_class = np.zeros((num_chars, 1)) 
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

    return loss, ps, hs, xs

#BACKWARD PROPAGATION ====================================

def backprop(ps, inputs, hs, xs):

    # make all zero matrices.
    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) 
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        # backprop into y. After taking the soft max in the input vector, 
        # subtract 1 from the value of the element corresponding to the correct label.
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
         # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dhraw = (1 - hs[t] * hs[t]) * dh 
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

# TRAINING ============================================

data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 


for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        
        inputs = [char_to_int[ch] for ch in txt_data[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in txt_data[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
        if (data_pointer+sequence_length+1 >= len(txt_data) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)
    
        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
        data_pointer += sequence_length # move data pointer
        
    if i % 100 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress

# PREDICTION ========================================

def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = ''.join(int_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

# PREDICTING ========================================
predict('h',30) # (char, len of output)
predict('e',30) # (char, len of output)
predict('l',30) # (char, len of output)
predict('o',30) # (char, len of output)


unique characters :  27
txt_data_size :  81
{'a': 0, 't': 1, 'z': 2, 'd': 3, 'f': 4, 'x': 5, 'b': 6, 'q': 7, 'p': 8, 'e': 9, 'k': 10, 's': 11, 'y': 12, 'v': 13, 'n': 14, 'h': 15, 'i': 16, 'u': 17, 'w': 18, 'm': 19, 'o': 20, 'g': 21, ' ': 22, 'r': 23, 'c': 24, 'l': 25, 'j': 26}
----------------------------------------------------
{0: 'a', 1: 't', 2: 'z', 3: 'd', 4: 'f', 5: 'x', 6: 'b', 7: 'q', 8: 'p', 9: 'e', 10: 'k', 11: 's', 12: 'y', 13: 'v', 14: 'n', 15: 'h', 16: 'i', 17: 'u', 18: 'w', 19: 'm', 20: 'o', 21: 'g', 22: ' ', 23: 'r', 24: 'c', 25: 'l', 26: 'j'}
----------------------------------------------------
[0, 6, 24, 3, 9, 4, 21, 15, 16, 26, 10, 25, 19, 14, 20, 8, 7, 23, 11, 1, 17, 13, 18, 5, 12, 2, 22, 0, 6, 24, 3, 9, 4, 21, 15, 16, 26, 10, 25, 19, 14, 20, 8, 7, 23, 11, 1, 17, 13, 18, 5, 12, 2, 22, 0, 6, 24, 3, 9, 4, 21, 15, 16, 26, 10, 25, 19, 14, 20, 8, 7, 23, 11, 1, 17, 13, 18, 5, 12, 2, 22]
----------------------------------------------------
data length :  81
(81, 27)
[[1 0 0